# Benchmarks

In [1]:
import numpy as np
import random
import math
import time
from shapely.geometry import Polygon, MultiPolygon, Point, box
from shapely.affinity import scale, translate
from shapely import wkt
from shapely.ops import transform
from pyproj import Transformer
# from functools import partial
import folium
from folium.features import DivIcon
from IPython.display import display, clear_output
# from branca.element import Figure

## Helper Functions

In [2]:
# from pyproj import Transformer
# from shapely.ops import transform
# 1326.748502108931 to 559.8474794921393
def calculateArea(poly_wkt):
    polygon = wkt.loads(poly_wkt)
    area_sq_degrees = polygon.area
    centroid = polygon.centroid
    lon, lat = centroid.x, centroid.y
    utm_zone = int((lon + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    # # Set up the transformation
    transformer = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    utm_polygon = transform(transformer.transform, polygon)
    area_sq_meters = utm_polygon.area
    area_sq_kilometers = area_sq_meters / 1_000_000  # Convert to square kilometers
    return area_sq_kilometers

In [3]:
# from shapely import wkt as shapely_wkt
# import shapely.geometry

def validate_ogc_polygon(wkt):
    """
    Validate that a polygon in WKT format is OGC compliant
    Args:
        wkt: WKT format polygon string
    Returns:
        tuple: (is_valid, message) where is_valid is a boolean and message describes any issues
    """
    try:
        # Parse the WKT string
        polygon = shapely_wkt.loads(wkt)
        
        # Check if the geometry is a polygon
        if not isinstance(polygon, shapely.geometry.Polygon):
            return False, "Not a polygon geometry"
        
        # Check if the polygon is valid according to OGC
        if not polygon.is_valid:
            reason = shapely.validation.explain_validity(polygon)
            return False, f"Invalid polygon: {reason}"
        
        # Check if the exterior ring is clockwise
        exterior_ring = polygon.exterior
        is_ext_clockwise = is_clockwise(exterior_ring.coords)
        if not is_ext_clockwise:
            return False, "Exterior ring is not clockwise (OGC requires clockwise)"
        
        # Check if interior rings (holes) are counterclockwise
        for interior in polygon.interiors:
            if is_clockwise(interior.coords):
                return False, "Interior ring (hole) is not counterclockwise (OGC requires counterclockwise)"
        
        # Check for self-intersections
        if not polygon.is_simple:
            return False, "Polygon has self-intersections"
        
        return True, "Polygon is OGC compliant"
        
    except ImportError:
        return False, "Validation requires shapely package"
    except Exception as e:
        return False, f"Validation error: {str(e)}"

def is_clockwise(coordinates):
    """
    Determine if a ring is clockwise by calculating the signed area
    Args:
        coordinates: List of (x, y) coordinates representing a ring
    Returns:
        boolean: True if clockwise, False if counterclockwise
    """
    # Shoelace formula to compute the signed area
    area = 0.0
    for i in range(len(coordinates) - 1):  # -1 because the last point is the same as the first
        j = (i + 1) % (len(coordinates) - 1)
        area += coordinates[i][0] * coordinates[j][1]
        area -= coordinates[j][0] * coordinates[i][1]
    
    # If area is positive, the ring is counterclockwise
    # If area is negative, the ring is clockwise
    return area < 0

def fix_ring_orientation(coordinates, should_be_clockwise=True):
    """
    Fix the orientation of a ring to be clockwise or counterclockwise
    Args:
        coordinates: List of (x, y) coordinates representing a ring
        should_be_clockwise: Boolean, True if the ring should be clockwise
    Returns:
        list: Coordinates in the correct orientation
    """
    is_clock = is_clockwise(coordinates)
    
    if (should_be_clockwise and not is_clock) or (not should_be_clockwise and is_clock):
        # Reverse the order of the coordinates, but keep the first/last point the same
        fixed = coordinates[:1] + coordinates[1:-1][::-1] + coordinates[-1:]
        return fixed
    
    return coordinates

def fix_ogc_polygon(wkt):
    """
    Fix common OGC compliance issues in a polygon
    Args:
        wkt: WKT format polygon string
    Returns:
        string: Fixed WKT polygon or the original if it cannot be fixed
    """
    try:        
        # Parse the WKT string
        polygon = wkt.loads(wkt)
        
        if not isinstance(polygon, Polygon):
            return wkt  # Can't fix non-polygons
        
        # Fix self-intersections and other validity issues
        if not polygon.is_valid:
            polygon = polygon.buffer(0)  # This often fixes invalid polygons
        
        # Fix ring orientations
        exterior_coords = list(polygon.exterior.coords)
        fixed_exterior = fix_ring_orientation(exterior_coords, True)  # Exterior should be clockwise
        
        fixed_interiors = []
        for interior in polygon.interiors:
            interior_coords = list(interior.coords)
            fixed_interior = fix_ring_orientation(interior_coords, False)  # Interior should be counterclockwise
            fixed_interiors.append(fixed_interior)
        
        # Reconstruct the WKT
        wkt = "POLYGON (("
        wkt += ", ".join([f"{p[0]} {p[1]}" for p in fixed_exterior])
        wkt += ")"
        
        for interior in fixed_interiors:
            wkt += ", ("
            wkt += ", ".join([f"{p[0]} {p[1]}" for p in interior])
            wkt += ")"
        wkt += ")"
        return wkt
    except Exception as e:
        print(f"Error fixing polygon: {e}")
        return wkt  # Return the original if we can't fix it
    

In [15]:
def displayOSM(i, polygon, bbox, area_percentage=None):
    # Create a map with the center based on the bounding box
    map_center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
    m = folium.Map(location=map_center, zoom_start=10) # SWEDEN ZOOM 4
    
    if isinstance(polygon, str):
        # Assuming the string is WKT format
        try:
            shape = wkt.loads(polygon)
            geo_json = shape.__geo_interface__
        except Exception as e:
            print(f"Error parsing polygon {e}")
    else:
        # Assuming it's already in a format folium can use
        geo_json = polygon
        
        # Add the polygon to the map with a unique style
    folium.GeoJson(
        geo_json,
        style_function=lambda x, color=f'#4400ff': {
                'fillColor': color,
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
        }
    ).add_to(m)

    north, south, east, west = bbox[3], bbox[1], bbox[0], bbox[2]
    # Add the bounding box to the map
    folium.Rectangle(bounds=[[north, west], [south, east]], color="red", fill=False).add_to(m)

    folium.map.Marker(
        [north-0.25, east],
        icon=DivIcon(
            icon_size=(350,56),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt; color: #4400ff;">points '+str(i)+' | area '+str(area_percentage*100.0)+'% BBOX</div>',
            )
        ).add_to(m)
    display(m)
    clear_output(wait=True)
    time.sleep(0.5)  # Pause to see the update
    # return m

## USER DEFINED GLOBAL VARIABLES

- https://observablehq.com/@jake-low/bounding-box-to-geojson-polygon

In [16]:
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305] # MUNICH
INPUT_BBOX_WKT = (box(*INPUT_BBOX, ccw=True)).wkt

## BM 1 - POLYGON WITH FIXED AREA and VARYING NUMBER OF POINTS

### Function

In [75]:
def genPolygon_AreaFIXED(INPUT_BBOX, num_points=None, area_percentage=None, circum_radius_factor=None):
    """
    Generate different polygons within the INPUT_BBOX in EPSG:4326 - WGS 84
    Ranges from small area polygon to large polygons with large area in OGC WKT format
    
    Args:
        INPUT_BBOX: List [min_lon, min_lat, max_lon, max_lat]
        area_percentage: Optional float between 0 and 1, representing what percentage of the bbox area 
                        the polygon should occupy. If None, a random value is used.
    Returns:
        String: WKT format polygon
    """
    min_lon, min_lat, max_lon, max_lat = INPUT_BBOX
          
    # Center of bbox
    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2

    # Anywhere of bbox
    # center_lon = random.uniform(min_lon, max_lon)
    # center_lat = random.uniform(min_lat, max_lat)

    # Get bbox dimensions
    bbox_width = max_lon - min_lon
    bbox_height = max_lat - min_lat
    # Calculate polygon dimensions based on area percentage For simplicity, we'll create a rectangular polygon
    scale_factor = math.sqrt(area_percentage)
    poly_width = bbox_width * scale_factor
    poly_height = bbox_height * scale_factor
    
    # Calculate polygon corners
    half_width = poly_width / 2
    half_height = poly_height / 2
    
    # Create a more interesting polygon by adding some random points and perturbations
    points = []
    
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        
        # Base radius is determined by the desired area
        x_radius = half_width
        y_radius = half_height
        
        # Add some randomness but maintain approximate area
        radius_factor = random.uniform(0.5, 1.0) # How uniform are the points (0.1 = random,  1.0 = uniform)
        
        x = center_lon + math.cos(angle) * x_radius * radius_factor
        y = center_lat + math.sin(angle) * y_radius * radius_factor
        
        # Ensure points are within the bbox
        x = max(min(x, max_lon), min_lon)
        y = max(min(y, max_lat), min_lat)
        
        points.append((x, y))
    
    # Close the polygon by adding the first point at the end
    points.append(points[0])
    
    # Convert to WKT format
    wkt = "POLYGON (("
    wkt += ", ".join([f"{p[0]} {p[1]}" for p in points])
    wkt += "))"

    # Double-check OGC compliance
    # is_valid, message = validate_ogc_polygon(wkt)
    # if not is_valid:
    #     # Try to fix the polygon
    #     wkt = fix_ogc_polygon(wkt)
    return wkt

### Map

In [87]:
## USER GLOBAL VARIABLES

NUM_OF_POINTS = 50
PERCENTAGE_OF_AREA = 0.25 # FIXED
CIRCUM_RADIUS_FACTOR = 0.5 # FIXED

for i in range(3, NUM_OF_POINTS, 5):
    polygon = genPolygon_AreaFIXED(INPUT_BBOX, i, area_percentage=PERCENTAGE_OF_AREA, circum_radius_factor=None)
    print(f"NUM_OF_POINTS: {i} \n\n{polygon}")
    displayOSM(i, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)
    # time.sleep(0.5) 

NUM_OF_POINTS: 48 

POLYGON ((11.593035906295775 48.15486111114964, 11.594682390806007 48.1584363995347, 11.594796769990307 48.162153592298665, 11.619471838888439 48.1713974081694, 11.619763583741069 48.17799691305197, 11.587075086569913 48.17269355015759, 11.59310373646065 48.18120520634404, 11.582670817264981 48.182192016656984, 11.585467926090734 48.19367999266679, 11.562349601304927 48.180228419279366, 11.562907477786876 48.195147694201545, 11.550520040200263 48.18840349766024, 11.541944444453677 48.180832571418705, 11.534364064775506 48.184510832588934, 11.521910421220833 48.19336233031984, 11.516405947331151 48.186610091586296, 11.502169693678702 48.190336474784, 11.511511030605842 48.175284554258475, 11.501255246073365 48.175813709855824, 11.47900090621217 48.179731947326644, 11.46684232476134 48.177189137128316, 11.49460990343992 48.16495738789419, 11.47802481761854 48.163680641208444, 11.470102940643747 48.159731495984516, 11.480345385921686 48.15486111114964, 11.4766705901099

## BM 2 - POLYGON WITH FIXED POINTS and VARYING AREA

In [90]:
## USER GLOBAL VARIABLES
NUM_OF_POINTS = 10
PERCENTAGE_OF_AREA = 0.25 #
UNIFORM_FACTOR = 0.5 

for PERCENTAGE_OF_AREA in np.arange(0.0, 1.02, 0.02):
    polygon = genPolygon_AreaFIXED(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
    print(f"Polygon Area: {calculateArea(polygon):.2f}km² ({(calculateArea(polygon)/calculateArea(INPUT_BBOX_WKT))*100.0:.2f})%| Area2(deg): {PERCENTAGE_OF_AREA*100.0:.2f}% \n\n{polygon}")
    displayOSM(NUM_OF_POINTS, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Polygon Area: 194.04km² (34.66)| Area2(%): 100.00% 

POLYGON ((11.669377308943432 48.15486111114964, 11.626449422783224 48.18647675981849, 11.592310086847469 48.23468209940104, 11.513699511040787 48.19962453301736, 11.441059375286919 48.19260500465459, 11.441513764547311 48.15486111114964, 11.399663725154605 48.101629960481006, 11.4970832774751 48.08376378145923, 11.573607700407225 48.10468022868004, 11.63952729701751 48.11835266817093, 11.669377308943432 48.15486111114964))


## BM 2x

In [91]:
# def get_bbox_polygon(bbox):
#     minx, miny, maxx, maxy = bbox
#     return box(minx, miny, maxx, maxy)

def create_regular_polygon(center_x, center_y, radius, num_points):
    """Generates a regular convex polygon"""
    return Polygon([
        (
            center_x + radius * math.cos(2 * math.pi * i / num_points),
            center_y + radius * math.sin(2 * math.pi * i / num_points)
        )
        for i in range(num_points)
    ])

def scale_to_bbox_area(geom, target_area):
    """Scales a geometry to match a target area"""
    current_area = geom.area
    scale_factor = math.sqrt(target_area / current_area)
    return scale(geom, xfact=scale_factor, yfact=scale_factor, origin='center')

# GENERATOR SUB FUNCTIONS

def genCase1(bbox, num_points, percentage_of_area=0.1):
    """Simple polygon with fixed area (50% of BBOX)"""
    bbox_poly = box(*INPUT_BBOX, ccw=True)
    cx, cy = bbox_poly.centroid.x, bbox_poly.centroid.y
    radius = min(bbox_poly.bounds[2] - bbox_poly.bounds[0],
                 bbox_poly.bounds[3] - bbox_poly.bounds[1]) / 4
    poly = create_regular_polygon(cx, cy, radius, num_points)
    poly_scaled = scale_to_bbox_area(poly, bbox_poly.area * percentage_of_area)
    return poly_scaled.wkt

def genCase2(bbox, num_points=4, percentage_of_area=1.0):
    """Simple polygon with varying area (1%-100%)"""
    return genCase1(bbox, num_points, percentage_of_area)

In [94]:
for PERCENTAGE_OF_AREA in np.arange(0.0, 1.02, 0.05):
    polygon = genCase2(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
    print(f"Polygon Area: {calculateArea(polygon):.2f}km² ({(calculateArea(polygon)/calculateArea(INPUT_BBOX_WKT))*100.0:.2f})%| Area2(deg): {PERCENTAGE_OF_AREA*100.0:.2f}% \n\n{polygon}")
    displayOSM(NUM_OF_POINTS, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

Polygon Area: 559.85km² (100.00)%| Area2(deg): 100.00% 

POLYGON ((11.693681987202439 48.15486111114964, 11.664702695222118 48.24405020099647, 11.588833923847737 48.29917208994746, 11.495054965059616 48.29917208994746, 11.419186193685228 48.24405020099647, 11.390206901704914 48.15486111114964, 11.419186193685228 48.06567202130282, 11.495054965059616 48.010550132351824, 11.588833923847737 48.010550132351824, 11.664702695222118 48.06567202130282, 11.693681987202439 48.15486111114964))


## BM 3 - POLYGON WITH VARYING POINTS, FIXED AREA, HOLES

### Function

In [85]:
def genPolygon_with_hole(INPUT_BBOX, num_points=None, area_percentage=None, hole_size=None):
    """
    Generate a polygon with one hole within the INPUT_BBOX in EPSG:4326 - WGS 84
    
    Args:
        INPUT_BBOX: List [min_lon, min_lat, max_lon, max_lat]
        num_points: Optional int, number of points in the outer polygon. If None, a random number is used.
        area_percentage: Optional float between 0 and 1, representing what percentage of the bbox area 
                        the outer polygon should occupy. If None, a random value is used.
        hole_size: Optional float between 0 and 1, representing the size of the hole relative to the outer polygon.
                  If None, a random value is used.
        
    Returns:
        String: WKT format polygon with a hole
    """
    min_lon, min_lat, max_lon, max_lat = INPUT_BBOX
    
    # Determine parameters if not specified
    if num_points is None:
        num_points = random.randint(6, 15)
    
    if area_percentage is None:
        area_percentage = random.uniform(0.4, 0.9)  # Larger area to accommodate a hole
    
    if hole_size is None:
        hole_size = random.uniform(0.2, 0.6)  # Size of hole relative to outer polygon
    
    # Generate outer polygon
    # We'll start with a base shape like a circle to ensure we can create a valid hole
    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2
    
    # Scale the bbox to the area percentage
    scale = math.sqrt(area_percentage)
    poly_width = (max_lon - min_lon) * scale
    poly_height = (max_lat - min_lat) * scale
    
    # Generate outer points with some randomness
    outer_points = []
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        
        # Add some randomness but maintain shape
        radius_factor = random.uniform(0.85, 1.15)
        
        x = center_lon + math.cos(angle) * (poly_width / 2) * radius_factor
        y = center_lat + math.sin(angle) * (poly_height / 2) * radius_factor
        
        # Ensure points are within the bbox
        x = max(min(x, max_lon), min_lon)
        y = max(min(y, max_lat), min_lat)
        
        outer_points.append((x, y))
    
    # Close the outer ring
    outer_points.append(outer_points[0])
    
    # Generate a hole (smaller similar shape)
    hole_points = []
    # Use a smaller number of points for the hole
    hole_num_points = max(3, num_points // 2)
    
    for i in range(hole_num_points):
        angle = 2 * math.pi * i / hole_num_points
        # Rotate the hole slightly to avoid point overlap
        angle += math.pi / hole_num_points
        
        # Use a smaller radius for the hole
        radius_factor = random.uniform(0.85, 1.15) * hole_size
        
        x = center_lon + math.cos(angle) * (poly_width / 2) * radius_factor
        y = center_lat + math.sin(angle) * (poly_height / 2) * radius_factor
        
        hole_points.append((x, y))
    
    # Close the hole ring
    hole_points.append(hole_points[0])
    
    # Ensure OGC compliance: exterior should be clockwise, interior counterclockwise
    if is_clockwise(outer_points):
        # Good, exterior is already clockwise
        pass
    else:
        # Fix exterior orientation
        outer_points = outer_points[:1] + outer_points[1:-1][::-1] + outer_points[-1:]
    
    if not is_clockwise(hole_points):
        # Good, interior is already counterclockwise
        pass
    else:
        # Fix interior orientation
        hole_points = hole_points[:1] + hole_points[1:-1][::-1] + hole_points[-1:]
    
    # Convert to WKT format
    wkt = "POLYGON (("
    wkt += ", ".join([f"{p[0]} {p[1]}" for p in outer_points])
    wkt += "), ("
    wkt += ", ".join([f"{p[0]} {p[1]}" for p in hole_points])
    wkt += "))"
    
    # Double-check OGC compliance
    # is_valid, message = validate_ogc_polygon(wkt)
    # if not is_valid:
    #     # Try to fix the polygon
    #     wkt = fix_ogc_polygon(wkt)
        
    return wkt

### MAP

In [86]:
for PERCENTAGE_OF_AREA in np.arange(0.0, 1.02, 0.05):
    polygon = genPolygon_with_hole(INPUT_BBOX, NUM_OF_POINTS) #(INPUT_BBOX, NUM_OF_POINTS, PERCENTAGE_OF_AREA)
    print(polygon)
    displayOSM(NUM_OF_POINTS, polygon, INPUT_BBOX, PERCENTAGE_OF_AREA)

POLYGON ((11.683708499292422 48.15486111114964, 11.645643096323298 48.11606457856954, 11.581349909367512 48.09241014254419, 11.49291860509995 48.0771634831422, 11.427581784272503 48.11207487823657, 11.392562272282238 48.15486111114964, 11.416062537442205 48.20195701277, 11.503174471361165 48.21630493316085, 11.580591397270227 48.21610996691928, 11.640849819831617 48.19186434647638, 11.683708499292422 48.15486111114964), (11.61618775542879 48.18263758636093, 11.512791963543227 48.201062841647946, 11.459756536394938 48.15486111114964, 11.518815704818572 48.11820598739983, 11.610272225569048 48.1292977992032, 11.61618775542879 48.18263758636093))
